 # <center>**Funciones para la API**

En este notebook se desarrollaran las pruebas de las funciones que son solicitadas en la API.

## Importaciones

In [1]:
import pandas as pd
import json
import ast
import warnings
warnings.filterwarnings("ignore")

## Extraccion de la informacion

Cargamos los 3 datasets ya limpios

In [2]:
steam = pd.read_csv('Output_Steam_limpia_eda.csv', index_col=0)
reviews = pd.read_csv('australian_user_reviews_limpia_eda.csv', index_col=0) #elimina la columna que se genera llamada unnamed:0
items = pd.read_csv('australian_user_items_limpia_eda.csv', index_col=0) 

In [3]:
steam.head()

,genres,title,id,developer,year_release
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018


In [4]:
reviews.head()

,recommend,user_id,sentiment_analysis,year_review
item_id,,,,
1250,True,76561197970982479,2,2011
22200,True,76561197970982479,1,2011
43110,True,76561197970982479,2,2011
251610,True,js41637,2,2014
227300,True,js41637,2,2013


In [5]:
items.head()

,item_id,item_name,playtime_forever,items_count,user_id,playtime_hours
0,10,Counter-Strike,6,277,76561197970982479,0.100000
1,20,Team Fortress Classic,0,277,76561197970982479,0.000000
2,30,Day of Defeat,7,277,76561197970982479,0.116667
3,40,Deathmatch Classic,0,277,76561197970982479,0.000000
4,50,Half-Life: Opposing Force,0,277,76561197970982479,0.000000


Realizamos un merge para poder realizar una sola tabla para las columnas necesarias en las funciones

In [6]:
df_merge = items.merge(reviews, on=('user_id', 'item_id'))
df_merge.head()


,item_id,item_name,playtime_forever,items_count,user_id,playtime_hours,recommend,sentiment_analysis,year_review
0,22200,Zeno Clash,271,277,76561197970982479,4.516667,True,1,2011
1,1250,Killing Floor,10006,277,76561197970982479,166.766667,True,2,2011
2,43110,Metro 2033,834,277,76561197970982479,13.900000,True,2,2011
3,227300,Euro Truck Simulator 2,551,888,js41637,9.183333,True,2,2013
4,239030,"Papers, Please",349,888,js41637,5.816667,True,2,2013


Para poder hacer un merge entre las tablas requerimos cambiar le nombre de una columna debido a que contienen la misma inforamcion pero tienen indicie diferente.

In [7]:
#Renombramos la columna "id" por "item_id" de la tabla "steam"
steam= steam.rename(columns={'id': 'item_id'})

In [8]:
steam.head()

,genres,title,item_id,developer,year_release
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018


Reazliamos el merge entre la tabla resultante anteior y la siguiente tabla

In [9]:
#Realizamos un merge entre el merge anterior y "user_games"
df_merge = df_merge.merge(steam, on='item_id')
#Visualizamos los cambios
df_merge.head()

,item_id,item_name,playtime_forever,items_count,user_id,playtime_hours,recommend,sentiment_analysis,year_review,genres,title,developer,year_release
0,22200,Zeno Clash,271,277,76561197970982479,4.516667,True,1,2011,Action,Zeno Clash,ACE Team,2009
1,22200,Zeno Clash,271,277,76561197970982479,4.516667,True,1,2011,Indie,Zeno Clash,ACE Team,2009
2,22200,Zeno Clash,240,942,seantheextraprawnsheepguy,4.000000,True,2,2011,Action,Zeno Clash,ACE Team,2009
3,22200,Zeno Clash,240,942,seantheextraprawnsheepguy,4.000000,True,2,2011,Indie,Zeno Clash,ACE Team,2009
4,22200,Zeno Clash,690,61,pipekissXD,11.500000,True,2,2013,Action,Zeno Clash,ACE Team,2009


Obtenemos una tabla con un total de 95508 filas x 16 columnas

Convertimos a string algunas columnas

In [10]:
df_merge['item_name'] = df_merge['item_name'].astype(str)
df_merge['developer'] = df_merge['developer'].astype(str)

In [11]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97524 entries, 0 to 97523
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             97524 non-null  int64  
 1   item_name           97524 non-null  object 
 2   playtime_forever    97524 non-null  int64  
 3   items_count         97524 non-null  int64  
 4   user_id             97524 non-null  object 
 5   playtime_hours      97524 non-null  float64
 6   recommend           97524 non-null  bool   
 7   sentiment_analysis  97524 non-null  int64  
 8   year_review         97524 non-null  int64  
 9   genres              97524 non-null  object 
 10  title               97524 non-null  object 
 11  developer           97524 non-null  object 
 12  year_release        97524 non-null  int64  
dtypes: bool(1), float64(1), int64(6), object(5)
memory usage: 9.0+ MB


Al analizar las funciones que son requeridas, las columnas necesarias para llevar a cabo estas son: "item_id", "playtime_forever", "user_id", "genre", "year_release", "recommend" y "developer", "year_review', se deciden eliminar las demas

In [12]:
columnas_a_eliminar = ["items_count", "title"]
df_merge = df_merge.drop(columnas_a_eliminar, axis=1)

In [13]:
df_merge.head()

,item_id,item_name,playtime_forever,user_id,playtime_hours,recommend,sentiment_analysis,year_review,genres,developer,year_release
0,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Action,ACE Team,2009
1,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Indie,ACE Team,2009
2,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Action,ACE Team,2009
3,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Indie,ACE Team,2009
4,22200,Zeno Clash,690,pipekissXD,11.500000,True,2,2013,Action,ACE Team,2009


Convertimos a parquet la tabla final


In [14]:
df_merge.to_parquet('Tabla_API.parquet')

In [15]:
Tabla_API= pd.read_parquet('Tabla_API.parquet')

In [16]:
Tabla_API.head()

,item_id,item_name,playtime_forever,user_id,playtime_hours,recommend,sentiment_analysis,year_review,genres,developer,year_release
0,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Action,ACE Team,2009
1,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Indie,ACE Team,2009
2,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Action,ACE Team,2009
3,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Indie,ACE Team,2009
4,22200,Zeno Clash,690,pipekissXD,11.500000,True,2,2013,Action,ACE Team,2009


Para la funcion 3 necesitaremos 'item_name','year_review', 'recommend', 'sentiment_analysis', asi que para optimizar el espacio en render creamos un parquet con esta informacion

In [17]:
funcion3= Tabla_API[['item_name','year_review', 'recommend', 'sentiment_analysis']]

In [18]:
funcion3 = funcion3.drop_duplicates(subset=['item_name', 'year_review', 'recommend','sentiment_analysis'])
funcion3 = funcion3.reset_index(drop=True)
funcion3.to_parquet('funcion3.parquet')

Para la funcion 4 necesitaremos 'developer','year_review', 'recommend', 'sentiment_analysis', asi que para optimizar el espacio en render creamos un parquet con esta informacion

In [19]:
funcion4= Tabla_API[['developer','year_review', 'recommend', 'sentiment_analysis']]

In [20]:
funcion4 = funcion4.drop_duplicates(subset=['developer','year_review', 'recommend', 'sentiment_analysis'])
funcion4 = funcion4.reset_index(drop=True)
funcion4.to_parquet('funcion4.parquet')

Para la funcion  necesitaremos 'developer', 'sentiment_analysis, asi que para optimizar el espacio en render creamos un parquet con esta informacion

In [21]:
funcion5= Tabla_API[['developer', 'sentiment_analysis']]

In [22]:
funcion4 = funcion4.drop_duplicates(subset=['developer','year_review', 'recommend', 'sentiment_analysis'])
funcion4 = funcion4.reset_index(drop=True)
funcion4.to_parquet('funcion5.parquet')

Ya con los archivos listos, procedemos construir las funciones

# Definicion de las funciones

# Funcion 1

Esta funcion devuelve el año con mas horas jugadas para el genero dicho.

In [23]:
def PlayTimeGenre(genero: str):
    # Filtrar el DataFrame por el género específico
  genero_data = Tabla_API[Tabla_API['genres'].str.contains(genero, case=False, na=False)]

  # Verificar si se encontraron datos para el género especificado
  if genero_data.empty:
      print(f"No hay datos para el género {genero}.")
      return None

  # Encontrar el año con más horas jugadas
  max_hours_row = genero_data.groupby('year_release')['playtime_forever'].sum().idxmax()

  # Imprimir el resultado
  result = f"El año con más horas jugadas para el género {genero}: {max_hours_row}"

  return result

In [24]:
print( PlayTimeGenre('Action'))

El año con más horas jugadas para el género Action: 2012


# Funcion 2

Esta funcion devuelve el usuario que acumula mas horas jugadas para el genero dado y un alista de la acumulacion de horas jugadas por año

In [25]:
def UserForGenre(genre):
   # Filtrar el DataFrame por el género específico
   genre_data = Tabla_API[Tabla_API['genres'].str.contains(genre, case=False, na=False)]

   # Verificar si se encontraron datos para el género especificado
   if genre_data.empty:
       print(f"No hay datos para el género {genre}.")
       return None

   # Agrupar por usuario y sumar las horas jugadas
   user_playtimes = genre_data.groupby('user_id')['playtime_forever'].sum()

   # Encontrar el usuario con más horas jugadas
   user_max_playtime = user_playtimes.idxmax()

   # Obtener la acumulación de horas jugadas por año
   playtime_by_year = genre_data.groupby('year_release')['playtime_forever'].sum().reset_index()

   # Crear un diccionario con los resultados
   result = {
       'Usuario_con_mas_horas_jugadas': user_max_playtime,
       'Horas_jugadas_por_año': playtime_by_year.to_dict('records')
   }

   return result


In [26]:
print( UserForGenre('Casual'))

{'Usuario_con_mas_horas_jugadas': 'tsunamitad', 'Horas_jugadas_por_año': [{'year_release': 1992, 'playtime_forever': 50}, {'year_release': 1993, 'playtime_forever': 51}, {'year_release': 1994, 'playtime_forever': 67}, {'year_release': 1996, 'playtime_forever': 968}, {'year_release': 1997, 'playtime_forever': 493}, {'year_release': 1998, 'playtime_forever': 251}, {'year_release': 1999, 'playtime_forever': 32739}, {'year_release': 2001, 'playtime_forever': 89}, {'year_release': 2002, 'playtime_forever': 88}, {'year_release': 2003, 'playtime_forever': 12}, {'year_release': 2004, 'playtime_forever': 1150}, {'year_release': 2006, 'playtime_forever': 2736}, {'year_release': 2007, 'playtime_forever': 26774}, {'year_release': 2008, 'playtime_forever': 1362}, {'year_release': 2009, 'playtime_forever': 22795}, {'year_release': 2010, 'playtime_forever': 50015}, {'year_release': 2011, 'playtime_forever': 672427}, {'year_release': 2012, 'playtime_forever': 838691}, {'year_release': 2013, 'playtime_

# Funcion 3

Esta funcion devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [27]:
def UsersRecommend(year: int) :
 # Filtrar el DataFrame por el año específico y las recomendaciones positivas o neutrales
 year_data = funcion3[(funcion3['year_review'] == year)]
 year_data1 = funcion3[(funcion3['recommend'] == True) & (funcion3['sentiment_analysis'].isin([2, 1]))]
 year = year_data1.groupby('item_name')['recommend'].sum().sort_values(ascending=False).head(3)
 result = pd.DataFrame(year).to_dict()
 # Convertir el resultado a una lista de diccionarios
 result = [{"Puesto " + str(i+1) : v} for i, v in enumerate(result['recommend'])]
 
 return result

In [28]:
print(UsersRecommend(2014))

[{'Puesto 1': 'Killing Floor'}, {'Puesto 2': 'Counter-Strike: Source'}, {'Puesto 3': "Garry's Mod"}]


# Funcion 4

Esta funcion devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [29]:
def UsersWorstDeveloper(year: int):
 
 # Filtrar el DataFrame por el año específico y las recomendaciones positivas o neutrales
 year_data = funcion4[(funcion4['year_review'] == year)]
 year_data1 = funcion4[(funcion4['recommend'] == False) & (funcion4['sentiment_analysis']== 0)]
 year= year_data1.groupby('developer')['recommend'].sum().sort_values(ascending=False).head(3)
 result = pd.DataFrame(year).to_dict()
 # Convertir el resultado a una lista de diccionarios
 result = [{"Puesto " + str(i+1) : v} for i, v in enumerate(result['recommend'])]
 return result


In [30]:
UsersWorstDeveloper(2012)

[{'Puesto 1': '10th Art Studio,Adventure Productions'},
 {'Puesto 2': 'Pieces Interactive'},
 {'Puesto 3': 'Rain Games'}]

# Funcion 5

Esta funcion, según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

In [31]:
def sentiment_analysis(empresa_desarrolladora: str) -> dict:
  # Crear un diccionario que mapee los valores '0', '1' y '2' a 'Negativo', 'Neutral' y 'Positivo'
  sentiment_mapping = {0: 'Negativo', 1: 'Neutral', 2: 'Positivo'}

  # Reemplazar los valores en la columna 'sentiment_analysis'
  funcion5['sentiment_analysis'] = funcion5['sentiment_analysis'].replace(sentiment_mapping)

  # Filtrar el DataFrame por la empresa desarrolladora
  df_filtered = funcion5[funcion5['developer'] == empresa_desarrolladora]

  # Verificar si se encontraron datos para la empresa desarrolladora
  if df_filtered.empty:
      print(f"No hay datos para la empresa {empresa_desarrolladora}.")
      return None

  # Agrupar por análisis de sentimiento y contar las filas
  df_grouped = df_filtered.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='Numero de registros')

  # Convertir el DataFrame agrupado en un diccionario con el nombre de la empresa desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor
  sentiment_analysis = {empresa_desarrolladora: df_grouped.set_index('sentiment_analysis')['Numero de registros'].to_dict()}

  return sentiment_analysis


In [32]:
print(sentiment_analysis('Ubisoft'))

{'Ubisoft': {'Negativo': 22, 'Neutral': 72, 'Positivo': 104}}
